In [4]:
import polars as pl
import pandas as pd
import os
import json
import numpy as np
from catboost import CatBoostClassifier, CatBoostRanker, Pool, sum_models
import gc
from ebrec.evaluation.metrics_protocols import *
from fastauc.fastauc.fast_auc import CppAuc
import argparse
import joblib


In [5]:
N_BATCH = 10
RANKER = True
model_path = '/home/ubuntu/experiments/batch_train_catboost'
dataset_path = '/mnt/ebs_volume/experiments/preprocessing_train_new'
catboost_params = {
    "iterations": 2421,
    "learning_rate": 0.061372161824290145,
    "rsm": 0.681769606695633,
    "reg_lambda": 0.4953354255208565,
    "grow_policy": "SymmetricTree",
    "bootstrap_type": "MVS",
    "subsample": 0.5108219602277233,
    "random_strength": 14.089062269780399,
    "fold_permutation_block": 39,
    "border_count": 34,
    "sampling_frequency": "PerTreeLevel",
    "score_function": "Cosine",
    "depth": 8,
    "mvs_reg": 0.0015341832942953422
}


In [7]:
with open(os.path.join(dataset_path, 'data_info.json')) as data_info_file:
        data_info = json.load(data_info_file)
        
models = []   
for batch in range(N_BATCH):
            if RANKER:
                model = CatBoostRanker(
                    **catboost_params, cat_features=data_info['categorical_columns'])
            else:
                model = CatBoostClassifier(**catboost_params, cat_features=data_info['categorical_columns'])
                
            model.load_model(model_path + f'/models/model_{batch}.cbm', format='cbm')
            models.append(model)

weights = [1/N_BATCH] * N_BATCH
model = sum_models(models, weights=weights,ctr_merge_policy='IntersectingCountersAverage')
joblib.dump(model, os.path.join(model_path, 'model.joblib'))

['/home/ubuntu/experiments/batch_train_catboost/model.joblib']

,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,trendiness_score_3d,weekday,hour,...,mean_JS_l_inf_user,std_JS_l_inf_user,mean_topic_model_cosine_l_inf_user,topics_cosine_l_inf_user,article_delay_hours_l_inf_article,mean_JS_l_inf_article,std_JS_l_inf_article,mean_topic_model_cosine_l_inf_article,topics_cosine_l_inf_article,endorsement_20h_articleuser_l_inf_articleuser
0,2,16.0,NaN,False,2,NaN,False,395,7,21,...,0.499210,0.453403,0.920897,0.647336,0.012048,0.245138,0.407771,0.882152,0.644699,1.0
1,2,16.0,NaN,False,2,NaN,False,72,7,21,...,0.088987,0.354481,0.882601,0.956521,0.024096,0.041950,0.195826,0.831786,0.952625,1.0
2,2,16.0,NaN,False,2,NaN,False,295,7,21,...,0.404086,0.386142,0.869966,0.777281,0.118280,0.245222,0.283392,0.777757,0.774115,1.0
3,2,16.0,NaN,False,2,NaN,False,282,7,21,...,0.390346,0.481760,0.913757,0.858693,0.023810,0.133343,0.306269,0.866335,0.855195,1.0
4,2,16.0,NaN,False,2,NaN,False,554,7,21,...,0.774656,0.964732,0.949907,0.320483,0.012048,0.218484,0.532947,0.876442,0.319178,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28420751,1,26.0,NaN,False,2,NaN,False,512,4,10,...,0.325236,0.514818,0.816969,0.853378,0.000000,0.081166,0.357610,0.745813,0.835396,1.0
28420752,1,26.0,NaN,False,2,NaN,False,243,4,10,...,0.609505,0.671289,0.789678,0.618911,0.023810,0.149527,0.339907,0.721788,0.605869,1.0
28420753,1,26.0,NaN,False,2,NaN,False,666,4,10,...,0.434137,0.545898,0.913322,0.741515,0.006061,0.116484,0.414622,0.830918,0.725890,1.0
28420754,1,26.0,NaN,False,2,NaN,False,344,4,10,...,0.471949,0.563019,0.876786,0.830871,0.985745,0.167889,0.467627,0.787589,0.813363,0.2
